In [ ]:
%load_ext autoreload
%autoreload 2

from NLP_Lib import *

import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import json

from official.nlp import optimization # for creating custom optimizer

In [ ]:
# Get the info of the model we want to load
PATH_TO_MODEL_INFO = os.path.join(
    PATH_TO_MODELS_DIRECTORY,
    "10",
    "16",
    "10:57:45")

# save_at_n.hf files are to big to push to github
PATH_TO_WEIGHTS_TO_LOAD = os.path.join(PATH_TO_MODEL_INFO, "save_at_3.h5")
PATH_TO_JSON_PARAMS = os.path.join(PATH_TO_MODEL_INFO, "training_params.json")

params_dict = json.load(open(PATH_TO_JSON_PARAMS))
print("Path to weights:", PATH_TO_WEIGHTS_TO_LOAD)

In [31]:
# Using the fetched model params and weights, load the model we want to predict with
optimizer = optimization.create_optimizer(
    init_lr=float(params_dict[INITIAL_LEARNING_RATE]),
    num_train_steps=int(params_dict[NUMBER_OF_TRAINING_STEPS]),
    num_warmup_steps=int(params_dict[NUMBER_OF_WARMUP_STEPS]),
    optimizer_type=params_dict[OPTIMIZER])

model = build_classifier_model(SMALL_BERT[0], SMALL_BERT[1], optimizer)
model.load_weights(PATH_TO_WEIGHTS_TO_LOAD)

In [28]:
# Load in the testing data tsv, use the loaded model to make
# predictions and save them to a .csv in the form kaggle defines
testing_df = pd.read_table(PATH_TO_TESTING_TSV)
X = np.array(testing_df[TEXT_COL])
preds = model.predict(X)

submission_df = pd.DataFrame(dict(
        Id=testing_df[ID_COL],
        Category=map_probs_to_class(preds)))

In [29]:
submission_df.to_csv(
    os.path.join(PATH_TO_MODEL_INFO, "kaggle_submission_e3.csv"),
    index=False) # ignore the row index column when writing to the csv